In [ ]:
from groq import Groq
import pandas as pd
import time
import json
import os

# Groq istemcisini başlat
client = Groq(
    api_key="your_api_key"  # Kendi Groq API anahtarınızı kullanın
)

# === MODEL ===
MODEL = "llama-3.3-70b-versatile" # Önerilen model

# === PROMPT OLUŞTURMA ===
def prompt_olusturSentiment(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki user ve bot görüşmesini mesajda botun kullanıcıya yansıtmadığı düşünceleride var bir düğün analizi uzmanı gibi analiz et analiz sonucunu yazmana gerek yok ve userın konuşma mesajlarındaki sentimentine göre sentiment  etiketini ver sadece,
sentiment: [Pozitif / Negatif / Nötr]
Sentimenti Tek tek mesajlardan değilde konuşmanın geneline bakarak değerlendir ama her mesajı analiz et.
Sentiment Etiket Üretme Mantığı: Göndericinin user olduğu mesajlarda tamamen olumlu kesin olarak pozitif anlam içeriyorsa pozitif etiketle,göndericininin user olduğu mesajlarda genel olarak net bir şekilde negatif durumlar varsa negatif etiketle,bunlar dışındakileri yani user bot konuşmaları genelinde net olarak pozitif değilse, net olarak negatif değilse nötr etiketle.
Cevabı Türkçe oluştur:Sadece sentiment=pozitif, sentiment=negatif veya sentiment=nötr gibi analiz sonucunun yorumunu yapma:

Mesaj: "{mesaj}"

"""

def promptOlusturKonu(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki user ve bot  görüşmesi mesajını bir düğün analizi uzmanı gibi analiz et analiz sonucunu yazmana gerek yok mesj boyunca konusulan ilgili konuları liste olarak ver:

Mesaj: "{mesaj}"

Konu: . Etkinlik Türüyle İlgili Kelimeler
(düğün, düğün salonu, düğün mekanı, kır düğünü, havuz başı düğün, otel düğünü
kına, kına gecesi, kına organizasyonu, kına mekanı
nişan, nişan organizasyonu, nişan mekanı, söz
sünnet, sünnet düğünü, sünnet organizasyonu
doğum günü, baby shower, mezuniyet, parti, after party
2. Hizmet Türüyle İlgili Kelimeler
organizasyon, organizasyon firması, düğün organizasyonu, kına organizasyonu
fotoğrafçı, düğün fotoğrafçısı, video, çekim, klip
davetiye, nikah şekeri, gelin arabası, gelin buketi, gelinlik, damatlık, saç-makyaj
orkestra, DJ, müzik grubu, catering, yemek, kokteyl, pasta
balayı, balayı oteli, balayı villası
3. Mekan ve Lokasyonla İlgili Kelimeler
il, ilçe, semt adı (ör: İstanbul, Beylikdüzü, Kadıköy, İzmir, Ankara, Bursa)
açık alan, kapalı alan, bahçe, salon, villa, tarihi mekan, sosyal tesis, otel, tekne, havuz başı
yakın, çevre, komşu ilçeler
4. Tarih ve Zamanla İlgili Kelimeler
tarih, gün, hafta sonu, hafta içi, yaz, kış, ay ismi (ör: Haziran, Temmuz)
5. Bütçe ve Kapasiteyle İlgili Kelimeler
kişi başı, toplam bütçe, fiyat, ücret, uygun fiyatlı, lüks, ekonomik
davetli sayısı, kapasite, küçük grup, kalabalık, az kişi, çok kişi
6. İkram ve Menüyle İlgili Kelimeler
yemekli, yemeksiz, kokteyl, pasta, meşrubat, açık büfe, içecek, ikram)

Türkçe olarak bir konu listesi üret, sadece listele, ve mesajda net olarak beliritlen konuyu seç mesaj dışı konular ekleme, yorum yapma: konu=[konu listesi] gibi sadece liste
"""

def prompt_olusturYanitladimi(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki bütün müşteri bot konuşmalarını analiz et ama analiz sonuçlarını yazmana gerek yok ve gelen konuşmalarda botun userın sorularını yanıtlayıp yanıtlamadığını kontrol et konuşma boyunca userdan sonraki ilk gönderici botsa cevaplandı kabul edebilrisin:

Konuşmalar: "{mesaj}"

Cevap formatı: [Evet / Hayır]
sadece evet hayir olarak tek bir cevap ver.
"""

# === JSON DOSYASINI OKU ===
def parse_json(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            conversations = json.load(f)
    except Exception as e:
        print(f"JSON dosyası okunurken hata oluştu: {e}")
        return pd.DataFrame()

    rows = []
    for convo in conversations:
        for msg in convo["messages"]:
            content = msg.get("content")
            if isinstance(content, dict):
                text = content.get("text", "")
            else:
                text = str(content) if content else ""
            if not text or not str(text).strip():
                continue
            sender = "bot" if msg["sender_id"] and msg["sender_id"].startswith("bf") else "user"
            rows.append({
                "conversation_id": convo["conversation_id"],
                "timestamp": msg["created_at"],
                "sender": sender,
                "text": text.strip()
            })
    return pd.DataFrame(rows)

# === LLM ÇAĞRISI ===
def llm_etiketle(mesaj):
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": mesaj}],
            max_tokens=2048
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Hata: {e}")
        return "HATA"

# === VERİYİ YÜKLE VE İŞLE ===
def main():
    # JSON dosyasını oku
    json_path = "last-500-conversation-dugunbuketi.json"
    df = parse_json(json_path).reset_index(drop=True)
    
    if df.empty:
        print("Hata: Veri yüklenemedi. İşlem sonlandırılıyor.")
        return
    
    # conversation_id, text, sender ve timestamp sütunlarını al
    df = df[["conversation_id", "text", "sender", "timestamp"]].head(171)

    # Aynı conversation_id'ye sahip mesajları birleştir ve konuşma numaralarını oluştur
    birlesik_konusmalar = df.groupby("conversation_id").apply(
        lambda x: "\n".join(f"Gönderici: {row['sender']} - Mesaj: {row['text']}" for _, row in x.sort_values("timestamp").iterrows())
    ).reset_index(name="birlesik_mesaj")

    # Konuşma numaralarını oluştur (Konuşma 1, Konuşma 2, ...)
    birlesik_konusmalar["konusma"] = [f"Konuşma {i+1}" for i in range(len(birlesik_konusmalar))]

    # Yalnızca konusma ve birlesik_mesaj sütunlarını tut
    df_sonuc = birlesik_konusmalar[["konusma", "birlesik_mesaj"]]
    df_sonuc.to_csv("manuelLabels.csv",index=False, encoding="utf-8")
    #Kullanıcı mesajları için sentiment ve category etiketlemesi
    
    df_sonuc["sentimen"] = df_sonuc["birlesik_mesaj"].apply(lambda x: llm_etiketle(prompt_olusturSentiment(x)))
    df_sonuc["category"] = df_sonuc["birlesik_mesaj"].apply(lambda x: llm_etiketle(promptOlusturKonu(x)))

    # answered etiketlemesi için birlesik_mesaj kullan
    df_sonuc["answered"] = df_sonuc["birlesik_mesaj"].apply(
        lambda x: llm_etiketle(prompt_olusturYanitladimi(x)) if x else "HATA"
    )

    # API çağrıları arasında gecikme
    time.sleep(0.5)

    # Sonuçları kaydet
    output_path = "llama-3.3-70b-versatile_output1.csv"
    df_sonuc.to_csv(output_path, index=False, encoding="utf-8")
    print(f" Etiketleme tamamlandı → {output_path} dosyasına kaydedildi.")

if __name__ == "__main__":
     main()

C:\Users\meteh\AppData\Local\Temp\ipykernel_6532\1855853058.py:126: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  birlesik_konusmalar = df.groupby("conversation_id").apply(


✅ Etiketleme tamamlandı → llama-3.3-70b-versatile_output1.csv dosyasına kaydedildi.


In [ ]:
from groq import Groq
import pandas as pd
import time
import json
import os

# Groq istemcisini başlat
client = Groq(
    api_key="your_api_key"  # Kendi Groq API anahtarınızı kullanın
)

# === MODEL ===
MODEL = "gemma2-9b-it" # Önerilen model

# === PROMPT OLUŞTURMA ===
def prompt_olusturSentiment(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki user ve bot görüşmesini mesajda botun kullanıcıya yansıtmadığı düşünceleride var bir düğün analizi uzmanı gibi analiz et analiz sonucunu yazmana gerek yok ve userın konuşma mesajlarındaki sentimentine göre sentiment  etiketini ver sadece,
sentiment: [Pozitif / Negatif / Nötr]
Sentimenti Tek tek mesajlardan değilde konuşmanın geneline bakarak değerlendir ama her mesajı analiz et.
Sentiment Etiket Üretme Mantığı: Göndericinin user olduğu mesajlarda tamamen olumlu kesin olarak pozitif anlam içeriyorsa pozitif etiketle,göndericininin user olduğu mesajlarda genel olarak net bir şekilde negatif durumlar varsa negatif etiketle,bunlar dışındakileri yani user bot konuşmaları genelinde net olarak pozitif değilse, net olarak negatif değilse nötr etiketle.
Cevabı Türkçe oluştur:Sadece sentiment=pozitif, sentiment=negatif veya sentiment=nötr gibi analiz sonucunun yorumunu yapma:

Mesaj: "{mesaj}"

"""

def promptOlusturKonu(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki user ve bot  görüşmesi mesajını bir düğün analizi uzmanı gibi analiz et analiz sonucunu yazmana gerek yok mesj boyunca konusulan ilgili konuları liste olarak ver:

Mesaj: "{mesaj}"

Konu: . Etkinlik Türüyle İlgili Kelimeler
(düğün, düğün salonu, düğün mekanı, kır düğünü, havuz başı düğün, otel düğünü
kına, kına gecesi, kına organizasyonu, kına mekanı
nişan, nişan organizasyonu, nişan mekanı, söz
sünnet, sünnet düğünü, sünnet organizasyonu
doğum günü, baby shower, mezuniyet, parti, after party
2. Hizmet Türüyle İlgili Kelimeler
organizasyon, organizasyon firması, düğün organizasyonu, kına organizasyonu
fotoğrafçı, düğün fotoğrafçısı, video, çekim, klip
davetiye, nikah şekeri, gelin arabası, gelin buketi, gelinlik, damatlık, saç-makyaj
orkestra, DJ, müzik grubu, catering, yemek, kokteyl, pasta
balayı, balayı oteli, balayı villası
3. Mekan ve Lokasyonla İlgili Kelimeler
il, ilçe, semt adı (ör: İstanbul, Beylikdüzü, Kadıköy, İzmir, Ankara, Bursa)
açık alan, kapalı alan, bahçe, salon, villa, tarihi mekan, sosyal tesis, otel, tekne, havuz başı
yakın, çevre, komşu ilçeler
4. Tarih ve Zamanla İlgili Kelimeler
tarih, gün, hafta sonu, hafta içi, yaz, kış, ay ismi (ör: Haziran, Temmuz)
5. Bütçe ve Kapasiteyle İlgili Kelimeler
kişi başı, toplam bütçe, fiyat, ücret, uygun fiyatlı, lüks, ekonomik
davetli sayısı, kapasite, küçük grup, kalabalık, az kişi, çok kişi
6. İkram ve Menüyle İlgili Kelimeler
yemekli, yemeksiz, kokteyl, pasta, meşrubat, açık büfe, içecek, ikram)

Türkçe olarak bir konu listesi üret, sadece listele, ve mesajda net olarak beliritlen konuyu seç mesaj dışı konular ekleme, yorum yapma: konu=[konu listesi] gibi sadece liste
"""

def prompt_olusturYanitladimi(mesaj):
    return f"""
Seninle Türkçe konuşuyorum. Aşağıdaki bütün müşteri bot konuşmalarını analiz et ama analiz sonuçlarını yazmana gerek yok ve gelen konuşmalarda botun userın sorularını yanıtlayıp yanıtlamadığını kontrol et konuşma boyunca userdan sonraki ilk gönderici botsa cevaplandı kabul edebilrisin:

Konuşmalar: "{mesaj}"

Cevap formatı: [Evet / Hayır]
sadece evet hayir olarak tek bir cevap ver.
"""

# === JSON DOSYASINI OKU ===
def parse_json(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            conversations = json.load(f)
    except Exception as e:
        print(f"JSON dosyası okunurken hata oluştu: {e}")
        return pd.DataFrame()

    rows = []
    for convo in conversations:
        for msg in convo["messages"]:
            content = msg.get("content")
            if isinstance(content, dict):
                text = content.get("text", "")
            else:
                text = str(content) if content else ""
            if not text or not str(text).strip():
                continue
            sender = "bot" if msg["sender_id"] and msg["sender_id"].startswith("bf") else "user"
            rows.append({
                "conversation_id": convo["conversation_id"],
                "timestamp": msg["created_at"],
                "sender": sender,
                "text": text.strip()
            })
    return pd.DataFrame(rows)

# === LLM ÇAĞRISI ===
def llm_etiketle(mesaj):
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": mesaj}],
            max_tokens=2048
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Hata: {e}")
        return "HATA"

# === VERİYİ YÜKLE VE İŞLE ===
def main():
    # JSON dosyasını oku
    json_path = "last-500-conversation-dugunbuketi.json"
    df = parse_json(json_path).reset_index(drop=True)
    
    if df.empty:
        print("Hata: Veri yüklenemedi. İşlem sonlandırılıyor.")
        return
    
    # conversation_id, text, sender ve timestamp sütunlarını al
    df = df[["conversation_id", "text", "sender", "timestamp"]].head(171)

    # Aynı conversation_id'ye sahip mesajları birleştir ve konuşma numaralarını oluştur
    birlesik_konusmalar = df.groupby("conversation_id").apply(
        lambda x: "\n".join(f"Gönderici: {row['sender']} - Mesaj: {row['text']}" for _, row in x.sort_values("timestamp").iterrows())
    ).reset_index(name="birlesik_mesaj")

    # Konuşma numaralarını oluştur (Konuşma 1, Konuşma 2, ...)
    birlesik_konusmalar["konusma"] = [f"Konuşma {i+1}" for i in range(len(birlesik_konusmalar))]

    # Yalnızca konusma ve birlesik_mesaj sütunlarını tut
    df_sonuc = birlesik_konusmalar[["konusma", "birlesik_mesaj"]]
    df_sonuc.to_csv("manuelLabels.csv",index=False, encoding="utf-8")
    #Kullanıcı mesajları için sentiment ve category etiketlemesi
    
    df_sonuc["sentimen"] = df_sonuc["birlesik_mesaj"].apply(lambda x: llm_etiketle(prompt_olusturSentiment(x)))
    df_sonuc["category"] = df_sonuc["birlesik_mesaj"].apply(lambda x: llm_etiketle(promptOlusturKonu(x)))

    # answered etiketlemesi için birlesik_mesaj kullan
    df_sonuc["answered"] = df_sonuc["birlesik_mesaj"].apply(
        lambda x: llm_etiketle(prompt_olusturYanitladimi(x)) if x else "HATA"
    )

    # API çağrıları arasında gecikme
    time.sleep(0.5)

    # Sonuçları kaydet
    output_path = "llmGemmaOutput1.csv"
    df_sonuc.to_csv(output_path, index=False, encoding="utf-8")
    print(f" Etiketleme tamamlandı → {output_path} dosyasına kaydedildi.")

if __name__ == "__main__":
     main()

C:\Users\meteh\AppData\Local\Temp\ipykernel_6532\3254660630.py:126: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  birlesik_konusmalar = df.groupby("conversation_id").apply(


Hata: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}
✅ Etiketleme tamamlandı → llmGemmaOutput1.csv dosyasına kaydedildi.


Kontrol etmek icin csv olusturma


In [15]:
import pandas as pd
import json
def parse_json(filepath):
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            conversations = json.load(f)
    except Exception as e:
        print(f"JSON dosyası okunurken hata oluştu: {e}")
        return pd.DataFrame()

    rows = []
    for convo in conversations:
        for msg in convo["messages"]:
            content = msg.get("content")
            if isinstance(content, dict):
                text = content.get("text", "")
            else:
                text = str(content) if content else ""
            if not text or not str(text).strip():
                continue
            sender = "bot" if msg["sender_id"] and msg["sender_id"].startswith("bf") else "user"
            rows.append({
                "conversation_id": convo["conversation_id"],
                "timestamp": msg["created_at"],
                "sender": sender,
                "text": text.strip()
            })
    return pd.DataFrame(rows)
# JSON dosyasını oku
json_path = "last-500-conversation-dugunbuketi.json"
df = parse_json(json_path).reset_index(drop=True)
    
    
# conversation_id, text, sender ve timestamp sütunlarını al
df = df[["conversation_id", "text", "sender", "timestamp"]].head(171)

# Aynı conversation_id'ye sahip mesajları birleştir ve konuşma numaralarını oluştur
birlesik_konusmalar = df.groupby("conversation_id").apply(
    lambda x: "\n".join(f"Gönderici: {row['sender']} - Mesaj: {row['text']}" for _, row in x.sort_values("timestamp").iterrows())
).reset_index(name="birlesik_mesaj")

# Konuşma numaralarını oluştur (Konuşma 1, Konuşma 2, ...)
birlesik_konusmalar["konusma"] = [f"Konuşma {i+1}" for i in range(len(birlesik_konusmalar))]

# Yalnızca konusma ve birlesik_mesaj sütunlarını tut
df_sonuc = birlesik_konusmalar[["konusma", "birlesik_mesaj"]]
df_sonuc
df_sonuc.to_csv("deneme.csv",index=False,encoding="utf-8")



C:\Users\meteh\AppData\Local\Temp\ipykernel_6532\3524240948.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  birlesik_konusmalar = df.groupby("conversation_id").apply(
